# Prepare notebook

In [1]:
from bs4 import BeautifulSoup
import googlesearch
import isbnlib
import json
import pandas as pd
import urllib
import wikipedia



In [2]:
API_key = ''

# Load data set, clean, & scrape web for additional information

In [3]:
books = pd.read_csv('./goodreads_library_export.csv')

In [4]:
books = books[['ISBN13', 'Title', 'Author', 'My Rating','Publisher',
       'Number of Pages', 'Year Published', 'Original Publication Year',
       'Date Added', 'Bookshelves', 'Exclusive Shelf',
       'Private Notes']]

books = books.loc[~(books['Exclusive Shelf'] == 'to-read')]
del books['Exclusive Shelf']

In [5]:
books['Date Added'] = pd.to_datetime(books['Date Added'])

In [6]:
books['recommended'] = books.Bookshelves.fillna(False).astype(bool)
del books['Bookshelves']

In [7]:
books['ISBN13'] = books['ISBN13'].apply(lambda s: s.replace('=','').replace('"', '').strip())

In [31]:
scraped_data = {}

for index, row in books.fillna('').iterrows():
    scraped_data[index] = {'isbn': None,
                           'categories': None,
                           'description': None}

    # Pull the Google Books information 
    try:
        base_api_link = "https://www.googleapis.com/books/v1/volumes?q=isbn:<isbn>&key="+API_key
        with urllib.request.urlopen(base_api_link.replace('<isbn>', row['ISBN13'])) as f:
            text = f.read()

        decoded_text = text.decode("utf-8")
        google_books_data = json.loads(decoded_text) # deserializes decoded_text to a Python object
        for identifier in google_books_data['items'][0]['volumeInfo']['industryIdentifiers']:
            if identifier['type'] == 'ISBN_13':
                scraped_data[index]['isbn'] = identifier['identifier']
                break
        scraped_data[index]['categories'] = google_books_data['items'][0]['volumeInfo']['categories']
        scraped_data[index]['description'] = google_books_data['items'][0]['volumeInfo']['description']
    except:
        try:
            base_api_link = "https://www.googleapis.com/books/v1/volumes?q=intitle:<title>+inauthor:<author>&key="+API_key
            with urllib.request.urlopen(base_api_link.replace('<title>', row['Title'].replace(' ', '+'))
                                                      .replace('<author>', row['Author'].replace(' ', '+'))) as f:
                text = f.read()

            decoded_text = text.decode("utf-8")
            google_books_data = json.loads(decoded_text) # deserializes decoded_text to a Python object
            for identifier in google_books_data['items'][0]['volumeInfo']['industryIdentifiers']:
                if identifier['type'] == 'ISBN_13':
                    scraped_data[index]['isbn'] = identifier['identifier']
                    break
            scraped_data[index]['categories'] = google_books_data['items'][0]['volumeInfo']['categories']
            scraped_data[index]['description'] = google_books_data['items'][0]['volumeInfo']['description']
        except:
            print('Unable to collect Google Books data for %s'%row['Title'])

Unable to collect Google Books data for People We Meet: Unforgettable Conversations
Unable to collect Google Books data for The Order of Time
Unable to collect Google Books data for Classic Irish Short Stories
Unable to collect Google Books data for The Complete Works of Edgar Allan Poe
Unable to collect Google Books data for The Weather Detective: Rediscovering Nature’s Secret Signs
Unable to collect Google Books data for White Fragility: Why It’s So Hard for White People to Talk About Racism
Unable to collect Google Books data for The Creativity Code: How AI Is Learning to Write, Paint and Think
Unable to collect Google Books data for The Secret Wisdom of Nature: Trees, Animals, and the Extraordinary Balance of All Living Things ― Stories from Science and Observation
Unable to collect Google Books data for The Laws of Human Nature
Unable to collect Google Books data for The Art of Mindful Living: How to Bring Love, Compassion, and Inner Peace Into Your Daily Life
Unable to collect Go

In [36]:
books = books[[ 'Title', 'Author', 'Publisher', 'My Rating','Number of Pages', 'Year Published', 
                    'Original Publication Year', 'Date Added', 'recommended']].join(pd.DataFrame(scraped_data).T)

In [37]:
books.count()

Title                        610
Author                       610
Publisher                    594
My Rating                    610
Number of Pages              584
Year Published               597
Original Publication Year    571
Date Added                   610
recommended                  610
isbn                         557
categories                   576
description                  557
dtype: int64

In [38]:
books.to_pickle('20191128_google_book_data.pkl')